In [235]:
from func import *
import numpy as np
from ngboost import NGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from matplotlib import *

In [236]:
mov_inter = pd.read_csv("asset/mov_inter1.2.csv")
y_train = mov_inter['boxoffice_num0']
mov_inter = mov_inter.drop(['name','boxoffice_num0','sales_share'],axis=1)
mov_inter

,nation,rep_nation,distributor,date3,month,year,sales0,num_of_screen0,num_of_screening0,nation2,...,gen13,gen14,gen15,gen16,gen17,gen18,a_avg_dir,r_avg_dir,a_avg_act,r_avg_act
0,한국,한국,(주)스마일이엔티,2019.5.29,5,2019,1.088259e+09,446,5998,한국,...,0,0,0,0,0,0,4.193333e+05,7.146667,1.134620e+06,7.202083
1,영국,"영국,미국",유니버설픽쳐스인터내셔널 코리아(유),2015.11.11,11,2015,1.430148e+10,1105,51940,기타,...,0,0,0,0,0,0,8.579925e+05,8.010000,1.267767e+06,7.820909
2,독일,독일,(주)영화사 진진,2019.9.12,9,2019,2.592648e+08,59,1482,기타,...,0,0,0,0,0,0,1.668550e+04,8.630000,2.379033e+04,8.341667
3,미국,미국,(주)제이앤씨미디어그룹,2018.1.31,1,2018,1.833626e+09,211,9844,미국,...,0,0,0,0,1,0,0.000000e+00,0.000000,2.588981e+06,8.140000
4,미국,미국,롯데쇼핑㈜롯데엔터테인먼트,2016.3.3,3,2016,1.170501e+09,380,9615,미국,...,0,0,0,0,0,0,1.176000e+06,8.076000,2.838555e+05,7.517583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,한국,한국,(주)씨제이이엔엠,2015.12.16,12,2015,3.946978e+10,1095,81042,한국,...,0,0,0,0,0,0,4.386000e+06,7.952500,2.954000e+06,7.880286
2056,한국,한국,(주)라이크콘텐츠,2016.3.10,3,2016,1.738441e+08,124,1444,한국,...,0,0,0,0,0,0,0.000000e+00,0.000000,3.178646e+05,8.541429
2057,한국,한국,(주)마인스 엔터테인먼트,2013.10.8,10,2013,1.589000e+08,179,2130,한국,...,0,0,0,0,0,0,1.218000e+03,8.020000,2.679018e+05,8.016250
2058,미국,미국,이십세기폭스코리아(주),2015.9.3,9,2015,5.023085e+08,272,4501,미국,...,0,0,0,0,0,0,6.400000e+04,7.770000,1.881589e+05,7.375750


In [237]:
# avg col 결측치
col_list = ['a_avg_dir', 'a_avg_act','r_avg_dir', 'r_avg_act']
nan_num = []
# column별 결측치 갯수
for col in col_list:
    cnt = 0
    for i in range(len(mov_inter)):
        if mov_inter.loc[i][col] == 0:
            cnt+=1
    print(col,": ",cnt)
    nan_num.append(cnt)
nan_num

a_avg_dir :  445
a_avg_act :  274
r_avg_dir :  431
r_avg_act :  252


[445, 274, 431, 252]

In [238]:
# 각 avg col 평균값
dir_a_tot_avg = sum(mov_inter['a_avg_dir'])/(len(mov_inter['a_avg_dir']) - nan_num[0])
act_a_tot_avg = sum(mov_inter['a_avg_act'])/(len(mov_inter['a_avg_act']) - nan_num[1])
dir_r_tot_avg = sum(mov_inter['r_avg_dir'])/(len(mov_inter['r_avg_dir']) - nan_num[2])
act_r_tot_avg = sum(mov_inter['r_avg_act'])/(len(mov_inter['r_avg_act']) - nan_num[3])

In [239]:
mov_inter

,nation,rep_nation,distributor,date3,month,year,sales0,num_of_screen0,num_of_screening0,nation2,...,gen13,gen14,gen15,gen16,gen17,gen18,a_avg_dir,r_avg_dir,a_avg_act,r_avg_act
0,한국,한국,(주)스마일이엔티,2019.5.29,5,2019,1.088259e+09,446,5998,한국,...,0,0,0,0,0,0,4.193333e+05,7.146667,1.134620e+06,7.202083
1,영국,"영국,미국",유니버설픽쳐스인터내셔널 코리아(유),2015.11.11,11,2015,1.430148e+10,1105,51940,기타,...,0,0,0,0,0,0,8.579925e+05,8.010000,1.267767e+06,7.820909
2,독일,독일,(주)영화사 진진,2019.9.12,9,2019,2.592648e+08,59,1482,기타,...,0,0,0,0,0,0,1.668550e+04,8.630000,2.379033e+04,8.341667
3,미국,미국,(주)제이앤씨미디어그룹,2018.1.31,1,2018,1.833626e+09,211,9844,미국,...,0,0,0,0,1,0,0.000000e+00,0.000000,2.588981e+06,8.140000
4,미국,미국,롯데쇼핑㈜롯데엔터테인먼트,2016.3.3,3,2016,1.170501e+09,380,9615,미국,...,0,0,0,0,0,0,1.176000e+06,8.076000,2.838555e+05,7.517583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,한국,한국,(주)씨제이이엔엠,2015.12.16,12,2015,3.946978e+10,1095,81042,한국,...,0,0,0,0,0,0,4.386000e+06,7.952500,2.954000e+06,7.880286
2056,한국,한국,(주)라이크콘텐츠,2016.3.10,3,2016,1.738441e+08,124,1444,한국,...,0,0,0,0,0,0,0.000000e+00,0.000000,3.178646e+05,8.541429
2057,한국,한국,(주)마인스 엔터테인먼트,2013.10.8,10,2013,1.589000e+08,179,2130,한국,...,0,0,0,0,0,0,1.218000e+03,8.020000,2.679018e+05,8.016250
2058,미국,미국,이십세기폭스코리아(주),2015.9.3,9,2015,5.023085e+08,272,4501,미국,...,0,0,0,0,0,0,6.400000e+04,7.770000,1.881589e+05,7.375750


In [240]:
# 결측치를 rating은 avg 0.7로, audience는 0.2로 채우기
train = mov_inter.copy()
nan_value = [0.3*dir_a_tot_avg, 0.3*act_a_tot_avg, 0.7*dir_r_tot_avg, 0.7*act_r_tot_avg]
col_list = ['a_avg_dir', 'a_avg_act','r_avg_dir', 'r_avg_act']
cnt = 0
for col in col_list:
    for i in range(len(train)):
        if train.loc[i, col] == 0:
            train.loc[i, col] = nan_value[cnt]
    cnt += 1

In [241]:
# 필요없는 column drop
train = train.drop(['rep_nation', 'nation', 'sales0','distributor'],axis=1)

In [242]:
L_encoder = LabelEncoder()
grd_e = L_encoder.fit(train["grade"])
grd_e.classes_ = ['0', '전체 관람가', '12세 관람가', '15세 관람가', '청소년 관람불가']
train["grade"] = L_encoder.transform(train["grade"])
nat_e = L_encoder.fit(train["nation2"])
nat_e.classes_ = ['한국', '미국', '기타']
train["nation2"] = L_encoder.transform(train["nation2"])
train

,date3,month,year,num_of_screen0,num_of_screening0,nation2,num_of_ch,major_distributor,grade,time,...,gen13,gen14,gen15,gen16,gen17,gen18,a_avg_dir,r_avg_dir,a_avg_act,r_avg_act
0,2019.5.29,5,2019,446,5998,0,5,0.0,3,102.0,...,0,0,0,0,0,0,4.193333e+05,7.146667,1.134620e+06,7.202083
1,2015.11.11,11,2015,1105,51940,2,6,1.0,3,148.0,...,0,0,0,0,0,0,8.579925e+05,8.010000,1.267767e+06,7.820909
2,2019.9.12,9,2019,59,1482,2,2,0.0,3,111.0,...,0,0,0,0,0,0,1.668550e+04,8.630000,2.379033e+04,8.341667
3,2018.1.31,1,2018,211,9844,1,4,0.0,3,130.0,...,0,0,0,0,1,0,2.257683e+05,5.498639,2.588981e+06,8.140000
4,2016.3.3,3,2016,380,9615,1,4,1.0,3,144.0,...,0,0,0,0,0,0,1.176000e+06,8.076000,2.838555e+05,7.517583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,2015.12.16,12,2015,1095,81042,0,0,1.0,2,124.0,...,0,0,0,0,0,0,4.386000e+06,7.952500,2.954000e+06,7.880286
2056,2016.3.10,3,2016,124,1444,0,0,0.0,2,109.0,...,0,0,0,0,0,0,2.257683e+05,5.498639,3.178646e+05,8.541429
2057,2013.10.8,10,2013,179,2130,0,0,0.0,3,96.0,...,0,0,0,0,0,0,1.218000e+03,8.020000,2.679018e+05,8.016250
2058,2015.9.3,9,2015,272,4501,1,4,1.0,3,98.0,...,0,0,0,0,0,0,6.400000e+04,7.770000,1.881589e+05,7.375750


In [244]:
# encoding int
a_col = ['a_avg_dir', 'a_avg_act']
for col in a_col:
    for i in range(len(train)):
        train.loc[i, col] = int(train.loc[i,col])

numpy.float64

In [232]:
train = train.drop("date3", axis=1)
train.to_csv("asset/new/train.csv")

In [233]:
train

,month,year,num_of_screen0,num_of_screening0,nation2,num_of_ch,major_distributor,grade,time,gen1,...,gen13,gen14,gen15,gen16,gen17,gen18,a_avg_dir,r_avg_dir,a_avg_act,r_avg_act
0,5,2019,446,5998,0,5,0.0,3,102.0,0,...,0,0,0,0,0,0,4.193333e+05,7.146667,1.134620e+06,7.202083
1,11,2015,1105,51940,2,6,1.0,3,148.0,0,...,0,0,0,0,0,0,8.579925e+05,8.010000,1.267767e+06,7.820909
2,9,2019,59,1482,2,2,0.0,3,111.0,1,...,0,0,0,0,0,0,1.668550e+04,8.630000,2.379033e+04,8.341667
3,1,2018,211,9844,1,4,0.0,3,130.0,0,...,0,0,0,0,1,0,2.257683e+05,5.498639,2.588981e+06,8.140000
4,3,2016,380,9615,1,4,1.0,3,144.0,0,...,0,0,0,0,0,0,1.176000e+06,8.076000,2.838555e+05,7.517583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2055,12,2015,1095,81042,0,0,1.0,2,124.0,0,...,0,0,0,0,0,0,4.386000e+06,7.952500,2.954000e+06,7.880286
2056,3,2016,124,1444,0,0,0.0,2,109.0,0,...,0,0,0,0,0,0,2.257683e+05,5.498639,3.178646e+05,8.541429
2057,10,2013,179,2130,0,0,0.0,3,96.0,0,...,0,0,0,0,0,0,1.218000e+03,8.020000,2.679018e+05,8.016250
2058,9,2015,272,4501,1,4,1.0,3,98.0,0,...,0,0,0,0,0,0,6.400000e+04,7.770000,1.881589e+05,7.375750


In [234]:
y_train.to_csv("asset/new/y_train.csv")